<a href="https://colab.research.google.com/github/arkwith7/aSSIST_ML/blob/main/LLM%ED%95%99%EC%8A%B5%EB%8D%B0%EC%9D%B4%ED%84%B0%EC%83%9D%EC%84%B1Genstruct%ED%85%8C%EC%8A%A4%ED%8A%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Genstruct 소개
고품질의 합성 명령 데이터를 생성하는 것은 중요한 과제입니다. 표준 접근 방식은 상황 내 학습과 명령 쌍 생성을 위한 대규모 언어 모델 프롬프트에 크게 의존합니다. 이는 품질, 다양성, 명시적인 추론 부족이라는 측면에서 한계가 있습니다.

이러한 순진한 프롬프트 접근 방식을 개선하기 위한 두 가지 이전 방법은 다음과 같습니다.
- 검색 증강 생성(RAG) 파이프라인은 Wikipedia와 같은 소스의 구절을 지침 쌍으로 변환합니다.
-  대신 [Ada-Instruct](https://arxiv.org/abs/2310.04484)는 프롬프트에 의존하는 대신 지침을 생성하도록 사용자 지정 모델을 교육합니다. 이는 메시지만 표시하는 것에 비해 품질과 다양성을 향상시킵니다. 또한 Ada-Instruct 논문의 저자는 단 10개의 예제로도 훈련을 수행할 수 있다는 사실을 발견했습니다.

Genstruct는 이러한 이전 접근 방식을 결합하고 확장하는 새로운 방법입니다. Ada-instruct와 마찬가지로 프롬프트에 의존하지 않고 맞춤형으로 훈련된 모델입니다. 그러나 Ada-Instruct는 근거 없는 생성에 크게 의존하므로 환각을 일으킬 수 있습니다. 이를 완화하기 위해 Genstruct는 RAG 방법과 같이 사용자가 제공한 컨텍스트를 기반으로 지침을 생성합니다.

또한 Genstruct는 직접적인 질문과 응답보다는 복잡한 질문 생성과 생성된 각 명령 쌍에 대한 다단계 추론에 중점을 두어 이전 작업을 뛰어넘습니다.

## 명령어 쌍 생성(Generating instruction pairs)
Ada-Instruct는 Mistral을 기반으로 훈련됩니다. 특히, 수학이 많이 사용되는 topcs로 추론을 개선하기 위해 [MetaMath-Mistral-7B](meta-math/MetaMath-Mistral-7B) 모델을 통해 훈련되었습니다.

다른 Mistral 모델과 마찬가지로 Huggingface Hub에서 다음과 같이 가져올 수 있습니다.

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

MODEL_NAME = 'NousResearch/Genstruct-7B'

model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, device_map='cuda', load_in_8bit=True)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

/home/user/.conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Loading checkpoint shards:  33%|███▎      | 1/3 [00:01<00:03,  1.75s/it]

Loading checkpoint shards:  67%|██████▋   | 2/3 [00:03<00:01,  1.72s/it]

Loading checkpoint shards: 100%|██████████| 3/3 [00:04<00:00,  1.64s/it]

Loading checkpoint shards: 100%|██████████| 3/3 [00:04<00:00,  1.66s/it]

Genstruct works by generating instructions and answers from a user-provided context and title. It utilizes a custom prompt format, as in the following example:
```
[[[Title]]] p-value
[[[Content]]] The p-value is used in the context of null hypothesis testing in order to quantify the statistical significance of a result, the result being the observed value of the chosen statistic T {\displaystyle T}.[note 2] The lower the p-value is, the lower the probability of getting that result if the null hypothesis were true. A result is said to be statistically significant if it allows us to reject the null hypothesis. All other things being equal, smaller p-values are taken as stronger evidence against the null hypothesis.

The following is an interaction between a user and an AI assistant that is related to the above text.

[[[User]]]
```

The model then completes from `[[[User]]]`, generating an instruction and a response.


To simplify its use, the Genstruct tokenizer includes a 'chat template'. It accepts a list containing a single dict, with members 'title' and 'content' - for the title and content of the context to generate from:

In [ ]:
msg =[{
    'title': 'p-value',
    'content': "The p-value is used in the context of null hypothesis testing in order to quantify the statistical significance of a result, the result being the observed value of the chosen statistic T {\displaystyle T}.[note 2] The lower the p-value is, the lower the probability of getting that result if the null hypothesis were true. A result is said to be statistically significant if it allows us to reject the null hypothesis. All other things being equal, smaller p-values are taken as stronger evidence against the null hypothesis."
}]
inputs = tokenizer.apply_chat_template(msg, return_tensors='pt').cuda()

Generation can then be performed with `model.generate()`, as follows (or with vllm or whaatever other pipeline you prefer):

In [ ]:
gen = tokenizer.decode(model.generate(inputs, max_new_tokens=512)[0]).split(tokenizer.eos_token)[0]
print(gen)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[[[Title]]] p-value
[[[Content]]] The p-value is used in the context of null hypothesis testing in order to quantify the statistical significance of a result, the result being the observed value of the chosen statistic T {\displaystyle T}.[note 2] The lower the p-value is, the lower the probability of getting that result if the null hypothesis were true. A result is said to be statistically significant if it allows us to reject the null hypothesis. All other things being equal, smaller p-values are taken as stronger evidence against the null hypothesis.

The following is an interaction between a user and an AI assistant that is related to the above text.

[[[User]]]  The share prices of two rival companies, A and B, have been monitored for many years, allowing a large number of data points for rigorous statistical analysis. This year's summer, which is known to affect share prices, had two distinct sub-periods, A and B, which were roughly equal in length. The company 'A's share price, 

Note that the model is optimized for single-paragraph extracts from Wikipedia articles. You may have varying luck with other input types.

## Filtering outputs using a reward model
The model may occasionally generate incorrect or improperly formatted output - the likelihood of this can be reduced with clever sampling methods, such as rejection sampling using a reward model, or even simple regex filtering.

For instance, we might consider `OpenAssistant/reward-model-deberta-v3-large-v2` as a reward model, and perform best-of-n sampling:

In [ ]:
import torch
from transformers import AutoModelForSequenceClassification

N = 4

rm_tokenizer = AutoTokenizer.from_pretrained('OpenAssistant/reward-model-deberta-v3-large-v2')
rm_model = AutoModelForSequenceClassification.from_pretrained('OpenAssistant/reward-model-deberta-v3-large-v2', torch_dtype=torch.bfloat16)

def extract_pair(resp):
    response = resp.split('[[[Content]]]')[1]
    inst, resp = resp.split('[[[User]]]')[:2]
    return inst.strip(), resp.strip()

def score(resp):
    inst, resp = extract_pair(resp.split(tokenizer.eos_token)[0])

    with torch.no_grad():
        inputs = rm_tokenizer(inst, resp, return_tensors='pt')
        score = float(rm_model(**inputs).logits[0].cpu())
        return score

gens = tokenizer.batch_decode(model.generate(inputs, max_new_tokens=256, num_return_sequences=N, do_sample=True))
print(max(gens, key=score))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[[[Title]]] p-value
[[[Content]]] The p-value is used in the context of null hypothesis testing in order to quantify the statistical significance of a result, the result being the observed value of the chosen statistic T {\displaystyle T}.[note 2] The lower the p-value is, the lower the probability of getting that result if the null hypothesis were true. A result is said to be statistically significant if it allows us to reject the null hypothesis. All other things being equal, smaller p-values are taken as stronger evidence against the null hypothesis.

The following is an interaction between a user and an AI assistant that is related to the above text.

[[[User]]]  Two medical procedures were compared by flipping 2 coins, procedure A assumed to be better and so it was labeled head, while procedure B was labeled as tail for a flip. The coins where then flipped 25 times, with the following results:[{'Tails', 12}, {'Heads', 13}]

Which procedure had better results with statistical signi